In [ ]:
import pandas as pd
import networkx as nx
import backboning as bb  # assuming backboning.py is in your PYTHONPATH
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [ ]:
NC_backbone = nx.read_graphml(r"/Users/iben/Desktop/Social_graphs/SocialGraphs/Project/MUSIC/artist_network_NCbackbone_3.graphml")